In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=44e7168ce9b24d5b90b529ae5d376af1e906996b0d04c2ed8c7069d7c1f530b4
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
# https://drive.google.com/file/d/17y6tyGP_4MoPsdpxOToNXSlsNN42S6mD/view?usp=share_link 

# Load Data

In [ ]:
# Helper Functions
def rolling_mean(df, col_name, window_size):
    rolling_mean = df[col_name].rolling(window_size).mean()
    df[f"Mean of {col_name} for {window_size} days"] = rolling_mean
    return df

def rolling_std(df, col_name, window_size):
    rolling_mean = df[col_name].rolling(window_size).std()
    df[f"Std of {col_name} for {window_size} days"] = rolling_mean
    return df

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col
from pyspark.pandas.window import Rolling
import pyspark.sql.functions as f

# create a SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "15g").appName("DSW_Final_Project").getOrCreate()

# mount your Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# set the file path
file_id = '17y6tyGP_4MoPsdpxOToNXSlsNN42S6mD'
url = f'https://drive.google.com/uc?id={file_id}'

# download the file from Google Drive to Colab
spark.sparkContext.addFile(url)
ts_file_path = '/content/drive/MyDrive/Data Science in the Wild - Final Project/BTC_Prices.csv'
tw_file_path = '/content/drive/MyDrive//Data Science in the Wild - Final Project/mbsa.csv'


# read the file into a Spark DataFrame
time_series_df = spark.read.format("csv").option("sep", "\t").option("header", "true").load(ts_file_path)

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


Mounted at /content/drive


# Tweet Sentiment Analysis

In [ ]:
import re    # RegEx for removing non-letter characters
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


# Function to remove URLs from text
def remove_urls(text):
     # Regular expression to match URLs
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_regex, '', text)

remove_urls_udf = udf(lambda x:remove_urls(x),StringType())

In [ ]:
from pyspark.sql.functions import col, to_date, unix_timestamp

tweets_spark_df = spark.read.format("csv").option("sep", ",").option("header", "true").load(tw_file_path)

print(tweets_spark_df)


#tweets_spark_df = tweets_spark_df.withColumn("user_followers",tweets_spark_df.user_followers.cast('int'))
#tweets_spark_df = tweets_spark_df.filter(tweets_spark_df.user_followers > 0)
tweets_spark_df = tweets_spark_df.filter(tweets_spark_df.text.isNotNull())
tweets_spark_df = tweets_spark_df.withColumn("Date", to_date(unix_timestamp('Date', 'yyyy-MM-dd').cast("timestamp"))).filter(col('Date').isNotNull())
# tweets_spark_df = tweets_spark_df.filter(col("date").between(startDate, endDate))
#tweets_spark_df = tweets_spark_df.withColumn("clean_text", remove_urls_udf(tweets_spark_df.text))
# tweets_spark_df.count()

DataFrame[Date: string, text: string, Sentiment: string]


In [ ]:
tweets_spark_df.show(100)

+----------+-------------------------------------+---------+
|      Date|                                 text|Sentiment|
+----------+-------------------------------------+---------+
|2019-05-27|                 È appena uscito u...| Positive|
|2019-05-27|                 Cardano: Digitize...| Positive|
|2019-05-27|                 Another Test twee...| Positive|
|2019-05-27|                 Current Crypto Pr...|     null|
|2019-05-27|                 Spiv (Nosar Baz):...|     null|
|2019-05-27|                 #btc inceldiği ye...| Positive|
|2019-05-27|                 @nwoodfine We hav...|     null|
|2019-05-27|                 @pedronauck como ...| Positive|
|2019-05-27|ブラジルはまぁ置いといてもドイツは...|     null|
|2019-05-27|                 CHANGE IS COMING....|     null|
|2019-05-27|                 One of the useful...|     null|
|2019-05-27|                 Invested my Life ...| Positive|
|2019-05-27|                 Bitcoin Price Hit...| Positive|
|2019-05-27|               【毎日プレゼント企画】|  

In [ ]:
# Select the first 1000 rows from tweets_spark_df
thousand = tweets_spark_df.limit(1000)


In [ ]:
thousand.select("Date").distinct().count()
#tweets_spark_df.select('Date').distinct().count()from pyspark.sql.functions import col, to_date

thousand = thousand.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))


In [ ]:
thousand = thousand.select('Date', 'text')
# selected_tweets_spark_df.show()

In [ ]:
from pyspark.sql.functions import col, to_date

thousand = thousand.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd "))


In [ ]:
from pyspark.sql.functions import min, max
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

thousand.agg(min('Date'), max('Date')).show(truncate=False)


+----------+----------+
|min(Date) |max(Date) |
+----------+----------+
|2019-05-27|2019-05-27|
+----------+----------+



In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

# Load the BERT sentiment analysis pipeline
sentiment_classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

def extract_sentiment(text):
    return sentiment_classifier(text, truncation=True, padding=True)[0]['score']

extract_sentiment_udf = udf(lambda x:extract_sentiment(x),StringType())

thousand = thousand.withColumn("sentiment_score", extract_sentiment_udf(thousand.text))

# Perform sentiment analysis on each row of text in the 'cleantext' column
# sentiment_predictions = sentiment_classifier(tweets_spark_df['clean_text'], truncation=True, padding=True)


In [ ]:
thousand = thousand.withColumn("sentiment_score", col('sentiment_score').cast('double'))
thousand = thousand.groupBy("Date").mean("sentiment_score")

In [ ]:
pyspark_pandas_thousand = thousand.to_pandas_on_spark()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:5249: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [ ]:
from pyspark.sql.functions import col, avg
from pyspark.sql.window import Window
from pyspark.sql.functions import stddev

def rolling_mean(df, col_name, window_size):
    window = Window.orderBy('Date').rowsBetween(-(window_size-1), 0)
    rolling_mean_col = avg(col(col_name)).over(window)
    df = df.withColumn(f"Mean of {col_name} for {window_size} days", rolling_mean_col)
    return df

def rolling_std(df, col_name, window_size):
    window = Window.orderBy('Date').rowsBetween(-(window_size-1), 0)
    rolling_std_col = stddev(col(col_name)).over(window)
    df = df.withColumn(f"Std of {col_name} for {window_size} days", rolling_std_col)
    return df


In [ ]:
pyspark_pandas_thousand = rolling_mean(thousand, "avg(sentiment_score)", 3)
pyspark_pandas_thousand = rolling_mean(thousand, "avg(sentiment_score)", 7)
pyspark_pandas_thousand = rolling_mean(thousand, "avg(sentiment_score)", 30)
pyspark_pandas_thousand = rolling_std(thousand, "avg(sentiment_score)", 3)
pyspark_pandas_thousand = rolling_std(thousand, "avg(sentiment_score)", 7)
pyspark_pandas_thousand = rolling_std(thousand, "avg(sentiment_score)", 30)

In [ ]:
pyspark_pandas_thousand.show(5)

+----------+--------------------+---------------------------------------+
|      Date|avg(sentiment_score)|Std of avg(sentiment_score) for 30 days|
+----------+--------------------+---------------------------------------+
|2019-05-27|  0.7721435754895211|                                   null|
+----------+--------------------+---------------------------------------+



In [ ]:
#len(pyspark_pandas_thousand.index)

In [ ]:
pyspark_pandas_thousand = spark.createDataFrame(pyspark_pandas_thousand)

TypeError: ignored

In [ ]:
pyspark_pandas_thousand.columns

['Date', 'avg(sentiment_score)', 'Std of avg(sentiment_score) for 30 days']

# Time Series

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

time_series_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("delimiter", ";")\
    .load("BTC_Prices.csv")

dollar_cols = ["Open*", "close", "High**", "low", "volume"]
for col in dollar_cols:
    time_series_df = time_series_df.withColumn(col, f.regexp_replace(col, "\$", ""))
    time_series_df = time_series_df.withColumn(col, f.regexp_replace(col, ",", ""))
    time_series_df = time_series_df.withColumn(col, time_series_df[col].cast("float"))

# convert PySpark DataFrame to Pandas DataFrame
pandas_time_series_df = time_series_df.to_pandas_spark()

# Rolling Means

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open*", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open*", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open*", 30)

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close**", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close**", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close**", 30)

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 30)

# Rolling STDs

pandas_time_series_df = rolling_std(pandas_time_series_df, "Open*", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Open*", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Open*", 30)

pandas_time_series_df = rolling_std(pandas_time_series_df, "Close**", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Close**", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Close**", 30)

pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 30)

# Avg Price and Previous Day Avg price
# pandas_time_series_df['Avg Price'] = (pandas_time_series_df['High'] + pandas_time_series_df['Low']) / 2
# pandas_time_series_df['Prev Day Avg Price'] = pandas_time_series_df['Avg Price'].shift(1)

#  Previous Day close price
pandas_time_series_df['Prev Day Close Price'] = pandas_time_series_df['Close**'].shift(1)
pandas_time_series_df['Next Day Close Price'] = pandas_time_series_df['Close**'].shift(-1)

AnalysisException: ignored

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

time_series_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("delimiter", ";")\
    .load("BTC_Prices.csv")

dollar_cols = ["Open", "High", "Low", "Close", "Volume", "marketCap", "timestamp"]
for col in dollar_cols:
    time_series_df = time_series_df.withColumn(col, f.regexp_replace(col, "\$", ""))
    time_series_df = time_series_df.withColumn(col, f.regexp_replace(col, ",", ""))
    time_series_df = time_series_df.withColumn(col, time_series_df[col].cast("float"))



# convert PySpark DataFrame to Pandas DataFrame
pandas_time_series_df = time_series_df.toPandas()

# Rolling Means

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Open", 30)

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Close", 30)

pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 3)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 7)
pandas_time_series_df = rolling_mean(pandas_time_series_df, "Volume", 30)

# Rolling STDs

pandas_time_series_df = rolling_std(pandas_time_series_df, "Open", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Open", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Open", 30)

pandas_time_series_df = rolling_std(pandas_time_series_df, "Close", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Close", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Close", 30)

pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 3)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 7)
pandas_time_series_df = rolling_std(pandas_time_series_df, "Volume", 30)

# Avg Price and Previous Day Avg price
# pandas_time_series_df['Avg Price'] = (pandas_time_series_df['High'] + pandas_time_series_df['Low']) / 2
# pandas_time_series_df['Prev Day Avg Price'] = pandas_time_series_df['Avg Price'].shift(1)

#  Previous Day close price
pandas_time_series_df['Prev Day Close Price'] = pandas_time_series_df['Close'].shift(1)
pandas_time_series_df['Next Day Close Price'] = pandas_time_series_df['Close'].shift(-1)

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
time_series_spark_df = spark.createDataFrame(pandas_time_series_df)

In [ ]:
startDate = "2017-01-01"
endDate = "2019-01-01"

In [ ]:
from pyspark.sql.functions import col, to_date, unix_timestamp
time_series_spark_df = time_series_spark_df.withColumn("Date", to_date(unix_timestamp(col('timestamp').cast("string"), 'MMM dd, yyyy').cast("timestamp"))).filter(col("Date").between(startDate, endDate))


#time_series_spark_df = time_series_spark_df.withColumn("Date", to_date(unix_timestamp('timestamp', 'MMM dd, yyyy').cast("timestamp"))).filter(col("Date").between(startDate, endDate))


In [ ]:
from pyspark.sql.functions import col
time_series_spark_df = time_series_spark_df.withColumn("Date", to_date(unix_timestamp('date', 'MMM dd, yyyy').cast("timestamp"))).filter(col("Date").between(startDate, endDate))

# clean_spark_df.show()
# clean_spark_df.agg({'Date': 'min'}).show()
# clean_spark_df.agg({'Date': 'max'}).show()

In [ ]:
joined_spark_df = time_series_spark_df.join(rw_es_tweets_spark_df,time_series_spark_df.Date ==  rw_es_tweets_spark_df.date,"inner").drop(rw_es_tweets_spark_df.date)

In [ ]:
joined_spark_df = joined_spark_df.na.drop("any")
joined_spark_df.count()

In [ ]:
input_features = [col.name for col in joined_spark_df.schema if col.name != 'Date' and col.name != 'Next Day Close Price']
print(len(input_features))

In [ ]:
assembler = VectorAssembler(
    inputCols=input_features,
    outputCol="features")

featurized_spark_df = assembler.transform(joined_spark_df)
featurized_spark_df

In [ ]:
featurized_spark_df.show(truncate=False)

+----------+--------------+--------------+--------------+--------------+---------------+-----------------+------------------------+------------------------+-------------------------+--------------------------+--------------------------+---------------------------+-------------------------+-------------------------+--------------------------+-----------------------+-----------------------+------------------------+-------------------------+-------------------------+--------------------------+------------------------+------------------------+-------------------------+--------------------+--------------------+--------------------+---------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------+--------------------------------------+---------------------------------------+-------------------------------------------------------------------------------------------------------------------------------

# XGBOOST



In [ ]:
!pip install --user virtualenv
!virtualenv xgboost_env
!python -m venv xgboost_env
!source /content/xgboost_env/bin/activate; pip install pyarrow pandas venv-pack xgboost
!source /content/xgboost_env/bin/activate; pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!source /content/xgboost_env/bin/activate; venv-pack -o xgboost_env.tar.gz

from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# Split dataset into train and test
test_spark_df, train_spark_df = featurized_spark_df.limit(int(featurized_spark_df.count()*.2)), featurized_spark_df.subtract(featurized_spark_df.limit(int(featurized_spark_df.count()*.2)))

In [ ]:
trainRows = train_spark_df.count()
print(f"Training Samples count : {trainRows}")
testRows = test_spark_df.count()
print(f"Testing Samples count : {testRows}")

In [ ]:
# This is an XGBoost extension for GPU acceleration. On CPU, the convention established by the spark-ml library is used, where all features are stored in a single dataframe column

from xgboost.spark import SparkXGBRegressor
spark_reg_estimator = SparkXGBRegressor(
  features_col='features',
  label_col='Next Day Close Price',
  num_workers=1,
  # use_gpu=True
)

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel, TrainValidationSplit, TrainValidationSplitModel
from multiprocessing.pool import ThreadPool

def _parallelFitTasks(est, train, eva, validation, epm):
    modelIter = est.fitMultiple(train, epm)

    def singleTask():
        index, model = next(modelIter)
        metric = eva.evaluate(model.transform(validation, epm[index]))
        return index, metric

    return [singleTask] * len(epm)

def minimum(a,b):
  return  b if a > b else a


class CustomTrainValidationSplit(TrainValidationSplit):
    def _fit(self, dataset):
      est = self.getOrDefault(self.estimator)
      epm = self.getOrDefault(self.estimatorParamMaps)
      numModels = len(epm)
      eva = self.getOrDefault(self.evaluator)
      tRatio = self.getOrDefault(self.trainRatio)
      seed = self.getOrDefault(self.seed)
      randCol = self.uid + "_rand"
      validation, train = df.limit(int(df.count()*(1-tRatio))), df.subtract(df.limit(int(df.count()*(1-tRatio))))    

      tasks = _parallelFitTasks(est, train, eva, validation, epm)
      pool = ThreadPool(processes= minimum(self.getParallelism(), numModels))
      metrics = [None] * numModels
      for j, metric in pool.imap_unordered(lambda f: f(), tasks):
          metrics[j] = metric
      train.unpersist()
      validation.unpersist()

      if eva.isLargerBetter():
          bestIndex = np.argmax(metrics)
      else:
          bestIndex = np.argmin(metrics)
      bestModel = est.fit(dataset, epm[bestIndex])
      return self._copyValues(TrainValidationSplitModel(bestModel, metrics))


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np

evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol=spark_reg_estimator.getLabelCol(),
                                predictionCol=spark_reg_estimator.getPredictionCol())

# Parameter grid search for xgboost


paramSearchGrid = (ParamGridBuilder()
             .addGrid(spark_reg_estimator.max_depth,[8])
             .addGrid(spark_reg_estimator.n_estimators, [100])
             .addGrid(spark_reg_estimator.gamma, [0.01])
             .addGrid(spark_reg_estimator.learning_rate, [0.01])
             .addGrid(spark_reg_estimator.base_score, [0.3, 0.5])
             .build())

# Grid search settings from the paper
# paramSearchGrid = (ParamGridBuilder()
#              .addGrid(spark_reg_estimator.max_depth,[2, 3, 4, 5, 6, 7, 8, 10])
#              .addGrid(spark_reg_estimator.n_estimators, [50, 100, 200])
#              .addGrid(spark_reg_estimator.gamma, [0, 0.001, 0.01, 0.1])
#              .addGrid(spark_reg_estimator.learning_rate, [0.001, 0.01, 0.1, 0.15, 0.2])
#              .addGrid(spark_reg_estimator.base_score, [0.3, 0.5, 0.7])
#              .build())


print("Number of combinations:", len(paramSearchGrid))


cv = CustomTrainValidationSplit(estimator=spark_reg_estimator, estimatorParamMaps=paramSearchGrid, evaluator=evaluator, trainRatio=0.8)
xgb_regressor_model = cv.fit(train_spark_df)

In [ ]:
# Print the best params
xgb_regressor_model.bestModel.extractParamMap()

In [ ]:
# Run model on test set
transformed_test_spark_dataframe = xgb_regressor_model.bestModel.transform(test_spark_df)

# Results & Plotting

In [ ]:
from pyspark.sql.functions import col

transformed_test_spark_dataframe.select(col("Date"), col("Next Day Close Price"), col("prediction")).show()

In [ ]:
from pyspark.sql.functions import *

ordered_transformed_test_spark_dataframe = transformed_test_spark_dataframe.orderBy(unix_timestamp(transformed_test_spark_dataframe["Date"], "MMM dd, yyyy").cast("timestamp"))
#unix_timestamp(transformed_test_spark_dataframe["Date"], "MMM dd, yyyy").cast("timestamp")

In [ ]:
ordered_transformed_test_spark_dataframe_pd = ordered_transformed_test_spark_dataframe.toPandas()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

renamed_ordered_transformed_test_spark_dataframe_pd = ordered_transformed_test_spark_dataframe_pd.rename(columns={"Next Day Close Price": "Actual", "prediction": "Prediction"})
plot_df = pd.melt(renamed_ordered_transformed_test_spark_dataframe_pd[["Date", "Prediction", "Actual"]], ['Date'], var_name="Next Day Close Price", value_name="BTC Price ($)")


In [ ]:
fig = plt.figure(figsize=(25,12))
ax = sns.lineplot(x='Date', y='BTC Price ($)', hue='Next Day Close Price', data=plot_df)
ax.tick_params(axis='x', labelrotation = 90)
plt.yticks(fontsize=15)
plt.xlabel('Date', fontsize=18)
plt.ylabel('BTC Next Day Close Price ($)', fontsize=18)
plt.legend(fontsize='xx-large')
plt.title("BTC Next Day Close Price ($)\nPredicted vs Actual\n", fontsize=22)

In [ ]:
model_feature_importance = xgb_regressor_model.bestModel.get_booster().get_score(importance_type='weight')
print(xgb_regressor_model.bestModel.get_booster())
print(model_feature_importance)
model_feature_importance_df = pd.DataFrame(model_feature_importance.items(), columns=['Feature_code', 'F-score'])
model_feature_importance_df['Feature_code'] = input_features
print(input_features)


In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
sns.barplot(y="Feature_code", x="F-score", data=model_feature_importance_df, orient='h')
# sns.despine(left=True, bottom=True)
plt.title("XGBoost Feature Importance", fontsize=16)
plt.ylabel('Features', fontsize=16)
plt.xlabel('F-score', fontsize=16)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="Next Day Close Price", predictionCol="prediction")

rsquared = evaluator.evaluate(transformed_test_spark_dataframe, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(transformed_test_spark_dataframe, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(transformed_test_spark_dataframe, {evaluator.metricName: "rmse"})

In [ ]:
print("Evaluation scores")
print("mae: ", mae)
print("rmse: ", rmse)
print("rsquared: ", rsquared)